## BERT Embedding 计算
本程序使用 BERT 计算文本的词向量，并输出为 NumPy 的 `.npy` 文件。

In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
import json

/opt/homebrew/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <0B637046-A38B-3A5C-80C6-E847C27DCCD5> /opt/homebrew/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <E3D17B4A-4867-3D49-BC92-E04C28EE0F45> /opt/homebrew/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


### 配置参数
设定输入文件、BERT 预训练模型、批处理大小等参数。

In [2]:
tokenized_jsonl = "/Users/jhx/Documents/Code/weibospider/output/comment_分词.jsonl"  # 分词后得到的JSONL
model_name = "bert-base-chinese"  # 预训练模型
output_file = "/Users/jhx/Documents/Code/weibospider/output/comment.npy"           # 嵌入向量保存路径
batch_size = 16                   # 批大小

### 读取 JSONL, 获取文本
本示例将 tokens 拼接成空格分隔的字符串后送入 BERT，也可使用原始句子。

In [3]:
sentences = []
with open(tokenized_jsonl, 'r', encoding='utf-8') as fin:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        record = json.loads(line)
        token_list = record.get("tokens", [])
        text = " ".join(token_list)
        sentences.append(text)

print(f"文本条数: {len(sentences)}")
if sentences:
    print(f"预览第一条: {sentences[0]}")

文本条数: 580
预览第一条: 庞氏骗局 时间


### 加载 BERT 预训练模型和 Tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

### 设置计算设备
自动检测是否有 GPU 可用。

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # 设置为评估模式

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### 切分批次
通过 DataLoader 处理数据，以便批量输入。

In [6]:
data_loader = DataLoader(sentences, batch_size=batch_size, shuffle=False)

### 计算 BERT Embedding 并提取 CLS 向量

In [7]:
cls_embeddings = []

for batch_sentences in tqdm(data_loader):
    inputs = tokenizer(
        batch_sentences,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    
    cls_batch = outputs.last_hidden_state[:, 0].cpu().numpy()
    cls_embeddings.append(cls_batch)

100%|██████████| 37/37 [00:22<00:00,  1.65it/s]


### 拼接所有批次的 CLS 向量

In [8]:
cls_embeddings_np = np.vstack(cls_embeddings)
print("最终生成的词向量", type(cls_embeddings_np), cls_embeddings_np.shape)

最终生成的词向量 <class 'numpy.ndarray'> (580, 768)


### 保存到 `.npy` 文件

In [9]:
np.save(output_file, cls_embeddings_np)
print(f"词向量存储于: {output_file}")

词向量存储于: /Users/jhx/Documents/Code/weibospider/output/comment.npy


### 测试加载 `.npy` 文件

In [10]:
embeddings = np.load(output_file)
print("加载回来，验证一下：", type(embeddings), embeddings.shape)

加载回来，验证一下： <class 'numpy.ndarray'> (580, 768)
